In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from weibull_model_football_scores.probs.weibull import weibull_likelihood

In [69]:
import pandas as pd
from weibull_model_football_scores.ingestion import load_data, load_prediction_dataset
from weibull_model_football_scores.model import WeibullCountFitter

from sklearn.metrics import precision_recall_fscore_support

In [51]:
df = load_data(league="seriea", season="22_23")

In [52]:
train = df[df["fixture"] < df["fixture"].max() - 3].copy()
test = df[df["fixture"] >= df["fixture"].max() - 3].copy()

In [54]:
w = WeibullCountFitter()

In [55]:
w.fit(train, learning_rate=0.01)

Starting likelihood: -749.0820457577335
Step 1, current likelihood: -708.0310183136045
Step 2, current likelihood: -687.2584045383283
Step 3, current likelihood: -670.2999617323087
Step 4, current likelihood: -656.4138969315983
Step 5, current likelihood: -645.1289538251566
Step 6, current likelihood: -636.0786658062285
Step 7, current likelihood: -628.9445536281493
Step 8, current likelihood: -623.4260717231252
Step 9, current likelihood: -619.2275118452685
Step 10, current likelihood: -616.0649505117524
Step 11, current likelihood: -613.6896843109588
Step 12, current likelihood: -611.905781077931
Step 13, current likelihood: -610.5676074693938
Step 14, current likelihood: -609.5672548145427
Step 15, current likelihood: -608.82348427789
Step 16, current likelihood: -608.2743430452693
Step 17, current likelihood: -607.872139747706
Step 18, current likelihood: -607.5799187556867
Step 19, current likelihood: -607.3690352445088
Step 20, current likelihood: -607.2174820728219
Step 21, curr

In [56]:
w.C_team.sort_values("alpha", ascending=False)

,team,alpha,beta
8,Napoli,1.680320,0.035298
10,Inter,1.122591,0.460647
7,Atalanta,1.115084,0.465301
2,Milan,0.973875,0.729060
3,Lazio,0.848351,0.312503
4,Juventus,0.759659,-0.110950
11,Udinese,0.729062,0.499002
12,Roma,0.625701,0.015697
6,Bologna,0.589335,0.811937
15,Monza,0.522905,0.762970


In [57]:
w.C_team.sort_values("beta", ascending=True)

,team,alpha,beta
4,Juventus,0.759659,-0.110950
12,Roma,0.625701,0.015697
8,Napoli,1.680320,0.035298
3,Lazio,0.848351,0.312503
10,Inter,1.122591,0.460647
5,Lecce,0.405732,0.461894
7,Atalanta,1.115084,0.465301
9,Torino,0.380563,0.489980
11,Udinese,0.729062,0.499002
18,Empoli,0.236458,0.618185


In [82]:
_test = test.copy()

In [85]:
test = test[test["fixture"] == test["fixture"].min()]

In [86]:
TRAIN = train.join(w.predict_under_over(train))
TEST = test.join(w.predict_under_over(test))

In [87]:
TRAIN["over_true"] = TRAIN["FTHG"] + TRAIN["FTAG"] > 2.5
TRAIN["over_pred"] = TRAIN["over"] > TRAIN["under"] 


TEST["over_true"] = TEST["FTHG"] + TEST["FTAG"] > 2.5
TEST["over_pred"] = TEST["over"] > TEST["under"] 

In [88]:
Z = T[T[["under", "over"]].max(axis=1) > 0.7]

In [89]:
precision_recall_fscore_support(TRAIN["over_true"], TRAIN["over_pred"])

(array([0.640625  , 0.61764706]),
 array([0.67768595, 0.57798165]),
 array([0.65863454, 0.5971564 ]),
 array([121, 109]))

In [90]:
precision_recall_fscore_support(TEST["over_true"], TEST["over_pred"])

(array([0.33333333, 0.25      ]),
 array([0.4, 0.2]),
 array([0.36363636, 0.22222222]),
 array([5, 5]))

In [91]:
TEST

,fixture,HomeTeam,AwayTeam,FTHG,FTAG,under,over,over_true,over_pred
230,23,Empoli,Napoli,0,2,0.444633,0.555367,False,True
231,23,Lecce,Sassuolo,0,1,0.589232,0.410768,False,False
232,23,Milan,Atalanta,2,0,0.299975,0.700025,False,True
233,23,Salernitana,Monza,3,0,0.361873,0.638127,True,True
234,23,Bologna,Inter,1,0,0.356339,0.643661,False,True
235,23,Udinese,Spezia,2,2,0.555997,0.444003,True,False
236,23,Verona,Fiorentina,0,3,0.646464,0.353536,True,False
237,23,Lazio,Sampdoria,1,0,0.682945,0.317055,False,False
238,23,Cremonese,Roma,2,1,0.634322,0.365678,True,False
239,23,Juventus,Torino,4,2,0.744959,0.255041,True,False


In [74]:
Z

,fixture,HomeTeam,AwayTeam,FTHG,FTAG,under,over,over_true,over_pred
232,23,Milan,Atalanta,2,0,0.299975,0.700025,False,True
239,23,Juventus,Torino,4,2,0.744959,0.255041,True,False
244,24,Roma,Juventus,1,0,0.797706,0.202294,False,False
258,25,Juventus,Sampdoria,4,2,0.806945,0.193055,True,False
259,25,Milan,Salernitana,1,1,0.288680,0.711320,False,True
267,26,Sampdoria,Verona,3,1,0.712719,0.287281,True,False


In [73]:
precision_recall_fscore_support(Z["over_true"], Z["over_pred"])

(array([0.25, 0.  ]),
 array([0.33333333, 0.        ]),
 array([0.28571429, 0.        ]),
 array([3, 3]))

In [15]:
1/w.predict_1x2_single_match("Napoli", "Milan")

pred
1    1.454842
2    7.478230
X    5.589125
Name: p, dtype: float64

In [31]:
1/(1 - (1/1.75 + 1/3.65))

6.468354430379745

In [ ]:
#cProfile.run('w.fit(df, n_iter=4)')

In [ ]:
X = load_prediction_dataset(league="seriea", season="22_23")

In [ ]:
preds = w.predict(X, how="under_over")

In [ ]:
X.join(preds)